# Getting available actions
Actions are configured in `app\configurations` directory. When the UI makes a request to get the actions available for an object, the `app/ajaxviews/actions.py` loads the yaml, and interprets what actions that the selected object is available to take. Those actions are sent to the user. 

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append("..")
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import sys, os


%load_ext lab_black

In [8]:
import app.ajaxviews.actions as actions
import app.models as models

c = models.CosmosdbClient()

You can load the full actions list. 

In [6]:
all_actions = actions.get_actions_config()
pd.DataFrame(all_actions)

,type,label,applies_to,effort,requires_attr,augments_self_properties,comment,leadingdesire
0,patriot_education,action,pop,10,{'factionLoyalty': 0.1},"{'factionLoyalty': 0.05, 'literacy': 0.01, 'ag...",expand public education programs with a collec...,NaN
1,safety_nets,action,pop,15,{'wealth': 0.1},"{'constitution': 0.05, 'wealth': '-.05'}",increase the general health of the population,NaN
2,individual_education,action,pop,10,{'wealth': 0.1},"{'conformity': -0.05, 'literacy': 0.1}",increase the general health of the population,NaN
3,patriot_propoganda,action,pop,10,NaN,"{'factionLoyalty': 0.1, 'conformity': 0.1, 'ag...","build national pride, increasin faction loyalt...",NaN
4,build_infrastructure,action,pop,20,{'wealth': 0.5},"{'wealth': -0.05, 'industrial_infrastructure': 1}",increase industry by investing in local infras...,wealth
5,healthcare_initialtives,action,pop,15,{'wealth': 0.1},"{'health': 0.01, 'wealth': '-.01'}",increase the health and literacy of other popu...,NaN
6,individual_education,action,pop,15,{'wealth': 0.1},"{'wealth': '-.1', 'literacy': 0.01, 'factionLo...","expand privatized education, increase literacy...",NaN


In [20]:
pop_id = "8880560711424"
c.run_query(f"g.V().has('objid','{pop_id}').valueMap()")
pop = c.clean_nodes(c.res)[0]
pop

{'isIdle': 'false',
 'name': 'Isgi Argorflo',
 'objid': '8880560711424',
 'conformity': 0.489,
 'literacy': 0.667,
 'aggression': 0.554,
 'constitution': 0.443,
 'health': 0.5,
 'isInFaction': '1234234304432',
 'industry': 0.4985,
 'wealth': 0.5828,
 'factionLoyalty': 0.459,
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '8880560711424'}

In [21]:
validator = actions.ActionValidator(pop, all_actions)


In [22]:
pd.DataFrame(validator.validate())

,type,label,applies_to,effort,requires_attr,augments_self_properties,comment,leadingdesire
0,patriot_education,action,pop,10,{'factionLoyalty': 0.1},"{'factionLoyalty': 0.05, 'literacy': 0.01, 'ag...",expand public education programs with a collec...,NaN
1,safety_nets,action,pop,15,{'wealth': 0.1},"{'constitution': 0.05, 'wealth': '-.05'}",increase the general health of the population,NaN
2,individual_education,action,pop,10,{'wealth': 0.1},"{'conformity': -0.05, 'literacy': 0.1}",increase the general health of the population,NaN
3,patriot_propoganda,action,pop,10,NaN,"{'factionLoyalty': 0.1, 'conformity': 0.1, 'ag...","build national pride, increasin faction loyalt...",NaN
4,build_infrastructure,action,pop,20,{'wealth': 0.5},"{'wealth': -0.05, 'industrial_infrastructure': 1}",increase industry by investing in local infras...,wealth
5,healthcare_initialtives,action,pop,15,{'wealth': 0.1},"{'health': 0.01, 'wealth': '-.01'}",increase the health and literacy of other popu...,NaN
6,individual_education,action,pop,15,{'wealth': 0.1},"{'wealth': '-.1', 'literacy': 0.01, 'factionLo...","expand privatized education, increase literacy...",NaN


## Creating a job

to create a job you need an agent and and action. This is done in the ajax views for that class (e.g. `app/ajaxviews/pop`) because they are a request from the UI.



In [24]:
def create_job(pop, action, universalTime):
    if type(universalTime) == list:
        universalTime = universalTime[0]
    time_to_complete = int(universalTime["currentTime"]) + int(action["effort"])
    # not actually creating the node here, just demo.
    # uid = create_action_node(c,action,pop)
    popToAction = {
        "node1": pop["objid"],
        "node2": "1234567890",
        "label": "takingAction",
        "name": "takingAction",
        "weight": time_to_complete,
        "actionType": action["type"],
        "status": "pending",
    }
    edges = [popToAction]
    return edges


time = {"currentTime": 10000}
create_job(pop, all_actions[0], time)

[{'node1': '8880560711424',
  'node2': '1234567890',
  'label': 'takingAction',
  'name': 'takingAction',
  'weight': 10010,
  'actionType': 'patriot_education',
  'status': 'pending'}]

## Resolving actions that are running